In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

In [3]:
celeba_bldr = tfds.builder('celeb_a')
celeba_bldr.download_and_prepare()
celeba_ds = celeba_bldr.as_dataset(shuffle_files=False)
celeba_train = celeba_ds['train']
celeba_val = celeba_ds['validation']
celeba_test = celeba_ds['test']

1598 examples [01:06, 188.89 examples/s]
11617 examples [01:06, 188.65 examples/s]
11636 examples [01:07, 188.48 examples/s]
11655 examples [01:07, 188.93 examples/s]
11675 examples [01:07, 190.49 examples/s]
11695 examples [01:07, 188.66 examples/s]
11714 examples [01:07, 189.05 examples/s]
11734 examples [01:07, 190.06 examples/s]
11754 examples [01:07, 191.27 examples/s]
11774 examples [01:07, 190.40 examples/s]
11794 examples [01:07, 189.43 examples/s]
11813 examples [01:08, 181.27 examples/s]
11832 examples [01:08, 181.79 examples/s]
11851 examples [01:08, 183.47 examples/s]
11870 examples [01:08, 180.61 examples/s]
11889 examples [01:08, 183.28 examples/s]
11909 examples [01:08, 185.47 examples/s]
11928 examples [01:08, 186.78 examples/s]
11947 examples [01:08, 181.37 examples/s]
11966 examples [01:08, 183.82 examples/s]
11986 examples [01:08, 185.85 examples/s]
12005 examples [01:09, 184.43 examples/s]
12025 examples [01:09, 186.37 examples/s]
12044 examples [01:09, 187.30 examp

In [4]:
celeba_train = celeba_train.take(16000)
celeba_val = celeba_val.take(1000)

In [5]:
def preprocess_augmentation(example, size=(64, 64), mode='train'):
    image = example['image']
    label = example['attributes']['Male']

    if mode == 'train':
        image_cropped = tf.image.random_crop(image, size=(178, 178, 3))
        image_resized = tf.image.resize(image_cropped, size=size)
        image_flipped = tf.image.random_flip_left_right(image_resized)
        return image_flipped/255.0, tf.cast(label, tf.int32) 
    else:
        image_cropped = tf.image.crop_to_bounding_box(image, offset_height=20, offset_width=0, target_height=178, target_width=178)
        image_resized = tf.image.resize(image_cropped,size=size)
        return image_resized/255.0, tf.cast(label, tf.int32)


In [6]:
tf.random.set_seed(42)
IMAGE_SIZE=(64,64)
celeba_train = celeba_train.map(lambda x: preprocess_augmentation(x, size=IMAGE_SIZE, mode='train'))
celeba_train = celeba_train.shuffle(buffer_size=2000).repeat().batch(32)

celeba_val = celeba_val.map(lambda x: preprocess_augmentation(x, size=IMAGE_SIZE, mode='eval'))
celeba_val = celeba_val.batch(32)

In [7]:
network = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Conv2D(64, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(rate=0.5),

    tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),

    tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='relu'),
    tf.keras.layers.GlobalAveragePooling2D(),

    tf.keras.layers.Dense(1, activation=None)
])
network.build(input_shape=(None, 64, 64, 3))
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 128)       7

In [8]:
network.compile(optimizer='adam', loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
history = network.fit(celeba_train, validation_data=celeba_val, epochs=20, steps_per_epoch=np.ceil(16000/32))
history.history

Epoch 1/20
500/500 [==============================] - 83s 161ms/step - loss: 0.6788 - accuracy: 0.5867 - val_loss: 0.5763 - val_accuracy: 0.6510
Epoch 2/20
500/500 [==============================] - 62s 125ms/step - loss: 0.5725 - accuracy: 0.6745 - val_loss: 0.4748 - val_accuracy: 0.7450
Epoch 3/20
500/500 [==============================] - 63s 125ms/step - loss: 0.5098 - accuracy: 0.7290 - val_loss: 0.4345 - val_accuracy: 0.7830
Epoch 4/20
500/500 [==============================] - 64s 128ms/step - loss: 0.4679 - accuracy: 0.7624 - val_loss: 0.3773 - val_accuracy: 0.8120
Epoch 5/20
500/500 [==============================] - 66s 133ms/step - loss: 0.4099 - accuracy: 0.7989 - val_loss: 0.3057 - val_accuracy: 0.8270
Epoch 6/20
500/500 [==============================] - 63s 125ms/step - loss: 0.3460 - accuracy: 0.8340 - val_loss: 0.2253 - val_accuracy: 0.8870
Epoch 7/20
500/500 [==============================] - 62s 124ms/step - loss: 0.2919 - accuracy: 0.8615 - val_loss: 0.1972 - val_ac

{'loss': [0.6556406617164612,
  0.5500884652137756,
  0.4859045147895813,
  0.45061248540878296,
  0.388178288936615,
  0.320389986038208,
  0.285805344581604,
  0.25936177372932434,
  0.24879103899002075,
  0.22759179770946503,
  0.21893200278282166,
  0.21614904701709747,
  0.20627431571483612,
  0.20249545574188232,
  0.19393771886825562,
  0.18910528719425201,
  0.1890449970960617,
  0.1823004186153412,
  0.17814023792743683,
  0.17514421045780182],
 'accuracy': [0.5980625152587891,
  0.6976875066757202,
  0.7483124732971191,
  0.7727500200271606,
  0.8106250166893005,
  0.8493750095367432,
  0.8676249980926514,
  0.8805624842643738,
  0.8884999752044678,
  0.8973125219345093,
  0.9048125147819519,
  0.9024375081062317,
  0.9091249704360962,
  0.9091874957084656,
  0.917062520980835,
  0.9185000061988831,
  0.9191874861717224,
  0.9195625185966492,
  0.9216874837875366,
  0.9209374785423279],
 'val_loss': [0.5763189196586609,
  0.4747571647167206,
  0.4344739019870758,
  0.37726959